In [14]:
import pandas as pd
import os
import sys
sys.path.append(os.path.abspath("../src"))

from data_cleaning_class import dataCleaning
df_chem = pd.read_csv("C:\\Users\\Aman\\Desktop\\week7\\data\\chemed123_messages.csv")
# df_doctor = pd.read_csv("C:\\Users\\Aman\\Desktop\\week7\\data\\doctorset_messages.csv")
df_chem = dataCleaning(df_chem)
# df_doctor = dataCleaning(df_doctor)


print(df_chem)


c:\Users\Aman\Desktop\week7\src\data_cleaning_class.py
True
